## Simulate Data
In this notebook, we simulate data using the registered device. Using the test data file, we feed data from one engine (unit 1) to the device in Cumulocity.

In [1]:
# Load required libraries
import json,requests,dateutil
from datetime import datetime,timedelta
import pandas as pd
from tqdm import tqdm
from requests.auth import HTTPBasicAuth
import json

In [ ]:
# Load config file with c8y credentials
configFile='../Data/CONFIG.json'
getConfig=json.loads(open(configFile,'r').read())

In [85]:
# Load data file
inputFile='../Data/test_FD001.csv'
data=pd.read_csv(inputFile)
data.columns=[ "unit", "cycle", "set_1", "set_2", "set_3", "s_1", "s_2", "s_3", "s_4", "s_5", "s_6", "s_7", "s_8", "s_9", "s_10", "s_11", "s_12", "s_13", "s_14", "s_15", "s_16", "s_17", "s_18", "s_19", "s_20", "s_21"]
print (data.shape)
data.head()
print(data.columns)

(13096, 26)
Index(['unit', 'cycle', 'set_1', 'set_2', 'set_3', 's_1', 's_2', 's_3', 's_4',
       's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13',
       's_14', 's_15', 's_16', 's_17', 's_18', 's_19', 's_20', 's_21'],
      dtype='object')


In [86]:
# Create a new data frame for the updated dataset
new_df = data.copy()

# Drop all engines except for unit 1
new_df = new_df[new_df['unit'] == 1]

# Drop unit and cycle columns
new_df.drop(['unit', 'cycle'], axis=1, inplace=True)

In [87]:
# Add a date column
now_dt = datetime.now()
new_df['index']=list(range(1,new_df.shape[0]+1))[::-1]
new_df['date']=new_df['index'].apply(lambda x: now_dt.date()-timedelta(days=x))

# Drop unneeded index column
new_df.drop(['index'], axis=1, inplace=True)

In [88]:
# Load data from data frame to the registered device
auth = HTTPBasicAuth(getConfig['c_user'], getConfig['c_pass'])
url = getConfig['c_url']+"/measurement/measurements"
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/vnd.com.nsn.cumulocity.measurement+json'
}

for i in tqdm(range(new_df.shape[0])):
    values=new_df.iloc[i]
    payload = json.dumps({
                        "s_1": {"F": {"unit": "NA","value": values['s_1']}},
                        "s_2": {"F": {"unit": "NA","value": values['s_2']}},
                        "s_3": {"F": {"unit": "NA","value": values['s_3']}},
                        "s_4": {"F": {"unit": "NA","value": values['s_4']}},
                        "s_5": {"F": {"unit": "NA","value": values['s_5']}},
                        "s_6": {"F": {"unit": "NA","value": values['s_6']}},
                        "s_7": {"F": {"unit": "NA","value": values['s_7']}},
                        "s_8": {"F": {"unit": "NA","value": values['s_8']}},
                        "s_9": {"F": {"unit": "NA","value": values['s_9']}},
                        "s_10": {"F": {"unit": "NA","value": values['s_10']}},
                        "s_11": {"F": {"unit": "NA","value": values['s_11']}},
                        "s_12": {"F": {"unit": "NA","value": values['s_12']}},
                        "s_13": {"F": {"unit": "NA","value": values['s_13']}},
                        "s_14": {"F": {"unit": "NA","value": values['s_14']}},
                        "s_15": {"F": {"unit": "NA","value": values['s_15']}},
                        "s_16": {"F": {"unit": "NA","value": values['s_16']}},
                        "s_17": {"F": {"unit": "NA","value": int(values['s_17'])}},
                        "s_18": {"F": {"unit": "NA","value": int(values['s_18'])}},
                        "s_19": {"F": {"unit": "NA","value": int(values['s_19'])}},
                        "s_20": {"F": {"unit": "NA","value": values['s_20']}},
                        "s_21": {"F": {"unit": "NA","value": values['s_21']}},
                        "time": str(values['date'])+'T'+ '00:00:00.000',
                        "source": {
                        "id": getConfig['c_device_source']
                          },
                          "type": "c8y_sensor"
                        })
    response = requests.post( url,auth=auth, headers=headers, data=payload)


100%|██████████| 31/31 [00:01<00:00, 24.71it/s]
